In [15]:
#%matplotlib qt
#%matplotlib inline

import numpy as np 
import pandas as pd 
import numpy.random as random
import random
from datetime import datetime

# Question 1:

In [3]:
lowercase = 'abcdefghijklmnopqrstuvwxyz'
sub_key = 'icznbkxgmprqtwfdyeoljvuahs'

In [4]:
def encrypt(msg, ref, key): 
    '''
    Parameters
    ----------
    msg : string of valid characters
    ref : string of N possible original characters
    key : string of N possible substitute characters 

    Returns
    -------
    Encrypted (substituted) message
    
    To Decrypt
    ----------
    Send key first and ref second, 
    i.e., encrypt(msg, key, ref)

    '''
    key_dict = dict(zip(ref, key))
    encryp = '' 
    for let in msg: 
        try: 
            encryp += key_dict[let]
        except: 
            encryp += let
    return encryp

In [5]:
bigrams_df = pd.read_csv('bigrams.csv', index_col=0) / 100000

In [6]:
encrypted = 'm it vbeh yjmbl. qbl lgb tfwlgo iwn hbieo zftb, '
encrypted = encrypted + 'lgbh ziw lirb wflgmwx keft tb, lgbh ziw lirb wflgmwx'
encrypted = encrypted + ' tfeb. m it of iqfwb, iwn of umlgfjl gfdb lgil m ziw'
encrypted = encrypted + ' zfwkefwl lgbt umlgfjl kbie. lgb qmkb lgil gio cfewb'
encrypted = encrypted + ' tb lgefjxg lgbob hbieo mo olmqq mw th giwno iwn th'
encrypted = encrypted + ' bhbo. ugblgbe m givb ojcnjbn ml, m rwfu wfl. cjl of'
encrypted = encrypted + ' qfwx io ml mo lgbeb ml umqq obbr mlo fuw uih fjl,'
encrypted = encrypted + ' gbbnqboo fk lgb umqq lgil mo umlgmw tb. gb kbqq mw'
encrypted = encrypted + ' fzlfcbe wmwblbbw bmxglbbw fw i nih lgil uio of yjmbl'
encrypted = encrypted + ' iwn olmqq fw lgb ugfqb kefwl lgil lgb ieth ebdfel'
encrypted = encrypted + ' zfwkmwbn mlobqk lf lgb omwxqb obwlbwzb iqq yjmbl fw'
encrypted = encrypted + ' lgb ubolbew kefwl. gb gin kiqqbw kfeuien iwn qih fw'
encrypted = encrypted + ' lgb bielg io lgfjxg oqbbdmwx. ljewmwx gmt fvbe fwb'
encrypted = encrypted + ' oiu lgil gb zfjqn wfl givb ojkkbebn qfwx; gmo kizb'
encrypted = encrypted + ' gin iw badeboomfw fk ziqt, io lgfjxg iqtfol xqin lgb'
encrypted = encrypted + ' bwn gin zftb.'

In [7]:
n_runs = 7
n_iters = 4000

In [8]:
def log_f(bgrm_df, msg):
    '''
    Parameters
    ----------
    bgrm_df : DataFrame with an index of bigrams 
              and a 'count' column.
    msg : A string of valid characters.

    Returns
    -------
    A log-score for the msg.

    '''    
    # Find pairs in the decrypted (using key) msg
    msg_df = pd.DataFrame() 
    for c1,c2 in zip(msg[:-1], msg[1:]): 
        if (not c1==' ') and (not c2==' '): 
            try: 
                msg_df.loc[c1+c2, 'count'] = msg_df.loc[c1+c2, 'count']+1
            except: 
                msg_df.loc[c1+c2, 'count'] = 1
                
    # For each bigram in the msg update log_f_score.    
    # If the bigram appears 0 times in the reference corpus,  
    #     subtract some small integer (log(k)<0 when k<1)          
    log_f_score = 0             
    for b in msg_df.index: 
        try: # succeed if bgrm_df.loc[b][0] > 0
            log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]
        except:
            log_f_score += -7 # bgrm_df.loc[b][0] = 0
            
    return log_f_score 

In [9]:
for _ in range(n_runs): 
    # Generate a random current substitution key
    # and initialize the current score to zero.
    cur_key = ''.join(rnd.sample(lowercase, len(lowercase)))
    cur_log_f = 0 
    
    start = datetime.now()
    for itr in range(n_iters):
        # Create a new key by swapping two random letters in the current key.
        c1, c2 = rnd.sample(cur_key, 2)
        new_key = cur_key.replace(c1,'*').replace(c2,c1).replace('*',c2) 
        
        # Decrypt with the new key and score the newly decrypted message.
        new_decrypted = encrypt(encrypted, new_key, lowercase)
        new_log_f = log_f(bigrams_df, new_decrypted) 
        
        # If new score > current score, accept the new key (current=new).
        # Else, with probability (new score)/(current score) accept the new key.
        if new_log_f > cur_log_f: # redundant condition: just for clarity
            cur_key = new_key
            cur_log_f = new_log_f
        else:
            r = random.uniform(0,1)
            if r < np.exp(new_log_f-cur_log_f): 
                cur_key = new_key
                cur_log_f = new_log_f
        if itr%500==0:       
            print('(%d) log(f) = %.2f , '%(itr,cur_log_f) + new_decrypted[:61] )
            print(datetime.now()-start)
            start = datetime.now()
            
# Print (or save) the current key and decrypted message.         
    print('------------------------------------------------------------')
    print('Proposed key:', cur_key, '\n')
    print(encrypt(encrypted, cur_key, lowercase))  
    print('------------------------------------------------------------\n\n')

/var/folders/wj/wzcmzk2n2232h1jj_2tfgl700000gn/T/ipykernel_31107/1851800651.py:29: RuntimeWarning: divide by zero encountered in log
  log_f_score += np.log(bgrm_df.loc[b][0])*msg_df.loc[b][0]


(0) log(f) = 0.00 , p wh dqus jxpqr. cqr roq hnvrom wvt sqwum fnhq, roqs fwv rwaq
0:00:00.096391
(500) log(f) = 5066.18 , e oh ksdw quest. cst trs haitrn oim wsodn fahs, trsw foi tovs
0:00:28.548861
(1000) log(f) = 5375.43 , i om kerl quiet. pet the manths ond leors came, thel con tove
0:00:28.579252
(1500) log(f) = 5431.46 , i am kerp qxiet. let the months and pears come, thep can tave
0:00:28.760825
(2000) log(f) = 5438.99 , i ay kerp quiet. let the yonths and pears coye, thep can tame
0:00:28.651652
(2500) log(f) = 5438.99 , i am kerp quiet. let the months and pears come, thep can taye
0:00:28.739492
(3000) log(f) = 5438.99 , a im kerp quaet. let the months ind peirs come, thep cin tiye
0:00:28.973250
(3500) log(f) = 5438.99 , i am kerp quiet. bet the months and pears come, thep can taye
0:00:29.156202
------------------------------------------------------------
Proposed key: icznbkxgmpvqtwfhyeoljduars 

i am kerp quiet. let the months and pears come, thep can taye nothing from me, 

**Best Scored Proposed key:** icznbkxgmpvqtwfhyeoljduars

i am kerp quiet. let the months and pears come, thep can taye nothing from me, thep can taye nothing more. i am so alone, and so without hove that i can confront them without fear. the life that has borne me through these pears is still in mp hands and mp epes. whether i hake subdued it, i ynow not. but so long as it is there it will seey its own wap out, heedless of the will that is within me. he fell in october nineteen eighteen on a dap that was so quiet and still on the whole front that the armp revort confined itself to the single sentence all quiet on the western front. he had fallen forward and lap on the earth as though sleeving. turning him oker one saw that he could not hake suffered long; his face had an exvression of calm, as though almost glad the end had come.

**Possible changes to the best scored substution:** \
p-->y\
y(old)-->k\
k(old)-->v\
v(old)-->p

i am very quiet. let the months and years come, they can take nothing from me, they can take nothing more. i am so alone, and so without hope that i can confront them without fear. the life that has borne me through these years is still in my hands and my eyes. whether i have subdued it, i know not. but so long as it is there it will seek its own way out, heedless of the will that is within me. he fell in october nineteen eighteen on a day that was so quiet and still on the whole front that the army revort confined itself to the single sentence all quiet on the western front. he had fallen forward and lay on the earth as though sleeving. turning him over one saw that he could not have suffered long; his face had an expression of calm, as though almost glad the end had come.


# Question 2

In [200]:
v =[99, 26, 71, 8, 28, 54, 5, 40, 100, 91, 27, 16, 78, 20, 97, 18, 63, 24, 1, 71]
w =[19, 17, 86, 52, 34, 58, 35, 96, 14, 92, 56, 51, 92, 70, 20, 83, 20, 18, 14, 25]
mw = 200 
b = 0.0005
d = 0.0005

i = 100000
r = 10

#c = choices
#b = beta
#v = values
#d = delta
#mw = maximum weight
#i = number of iterations
#r = number of runs

#log of e to the power of beta times total value of chosen item
def log_f(choices, values, beta):
    return beta * np.dot(choices,values)

In [201]:
def knapsack(values, weights, max_weight, number_of_runs, number_of_itr, beta = 0.0005, delta = 0.0005):
    
    for run in range(number_of_runs):
        
        cur_choice = np.random.randint(2, size = 20)
        cur_score = 0
        
        used_keys = []
        for itr in range(number_of_itr):
            
            #switch a choice of randomly sampled index
            new_choice = cur_choice
            index = random.randint(0, 19)
            if cur_choice[index] == 1:
                new_choice[index] = 0
            else:
                new_choice[index] = 1 
                
            #check if the weights are within the allowed limit
            weight_choices = []
            indexes = np.arange(20)
            for i in range(0, 19):
                if new_choice[i] == 1:
                    weight_choices.append(weights[i]) # listede saklama memory yer
                else:
                    pass
            weights_sum = sum(weight_choices)
            if weights_sum <= max_weight:
                pass
            else:
                continue
            
            if itr % 500 == 0:
                beta += delta
            
            new_score = log_f(new_choice, values, beta) 
            
            if new_score > cur_score:
                potential_choices = new_choice
                cur_score = new_score
            elif random.random() < new_score / cur_score:
                potential_choices = new_choice
                cur_score = new_score
            else:
                pass
            
    return potential_choices           

In [202]:
save = knapsack(v, w, mw, r, i)
save

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [203]:
indeces = np.where(save > 0)[0].tolist()
used_values = [v[i] for i in indeces]
sum(used_values)

653

# Question 3

In [ ]:
#here you can just follow the logic of question 1 - create a random string of 5 or 6 characters long, then randomly replace a letter 
#and score the new string. Repeat for a long long time until you get your final string, which *should* look like a name

In [209]:
lowercase = 'abcdefghijklmnopqrstuvwxyz'

In [210]:
trigrams = pd.read_csv('names_trigrams.csv')
trigrams.columns = ['Trigrams', 'Count']

In [211]:
n_runs = 7
n_iters = 4000

In [212]:
def log_f(trigram_df, name):
    '''
    Parameters
    ----------
    trigram_df : DataFrame with an index of trigrams 
                 and a 'count' column.
    name       : A string of valid characters.

    Returns
    -------
    A log-score for the name.

    '''    
    # Find pairs in the decrypted (using key) name
    name_df = pd.DataFrame() 
    for c1,c2,c3 in zip(name[:-1], name[1:], name[2:]): 
        if (not c1==' ') and (not c2==' ') and (not c3==' '): 
            try: 
                name_df.loc[c1+c2+c3, 'count'] = name_df.loc[c1+c2+c3, 'count']+1
            except: 
                name_df.loc[c1+c2+c3, 'count'] = 1
    
    # For each trigram in the name update log_f_score.    
    # If the trigram appears 0 times in the reference corpus,  
    #     subtract some small integer (log(k)<0 when k<1)          
    log_f_score = 0
    
    for b in name_df.index: 
        try: # succeed if name_df.loc[b][0] > 0
            log_f_score += np.log(np.where(trigrams["Trigrams"] == b)[0][0])*name_df.loc[b][0]
        except:
            log_f_score += -7 # name_df.loc[b][0] = 0
            
    return log_f_score 

In [215]:
for _ in range(n_runs): 
    # Generate a random current substitution key
    # and initialize the current score to zero.
    cur_key = ''.join(rnd.sample(lowercase, 6))
    cur_score = 0 
    
    for itr in range(n_iters):
        # Create a new key by swapping two random letters in the current key.
        c1 = ''.join(rnd.sample(cur_key, 1))
        t1 = ''.join(rnd.sample(lowercase, 1))
        new_key = cur_key.replace(c1, t1) 
        
        new_score = log_f(trigrams, new_key) 
        
        # If new score > current score, accept the new key (current=new).
        # Else, with probability (new score)/(current score) accept the new key.
        if new_score > cur_score: # redundant condition: just for clarity
            potential_name = new_key
            cur_score = new_score
        else:
            r = random.uniform(0,1)
            if r < np.exp(new_score - cur_score): 
                potential_name = new_key
                cur_score = new_score  
        
# Print (or save) the current key and decrypted message.         
    print('------------------------------------------------------------')
    print('Proposed name:', potential_name, ' Score:', cur_score, '\n')
    print('------------------------------------------------------------\n\n')

------------------------------------------------------------
Proposed name: yilboe  Score: 16.036507458901784 

------------------------------------------------------------


------------------------------------------------------------
Proposed name: cpliyo  Score: 17.26886731240527 

------------------------------------------------------------


------------------------------------------------------------
Proposed name: rvaija  Score: 30.427288547599638 

------------------------------------------------------------


------------------------------------------------------------
Proposed name: tigidz  Score: 15.86499046454815 

------------------------------------------------------------


------------------------------------------------------------
Proposed name: mniaws  Score: 30.530317446292337 

------------------------------------------------------------


------------------------------------------------------------
Proposed name: fbvhah  Score: 2.216778138136835 

----------------